In [8]:
# CARGA DE DATOS 
# Cargamos los CSV desde la carpeta /data/raw.

import pandas as pd

df_pm10 = pd.read_csv('../data/raw/pm 10 coyhaique (extended).csv', sep=';', encoding='latin1', engine='python')
df_pm25 = pd.read_csv('../data/raw/pm 2.5 coyhaique.csv', sep=';')
df_co = pd.read_csv('../data/raw/CO coyhaique.csv', sep=';')
df_so2 = pd.read_csv('../data/raw/SO2 coyahique.csv', sep=';')
df_meteo = pd.read_csv('../data/raw/export (extended).csv')
df_umbrales = pd.read_csv('../data/raw/umbrales_pm10.csv', sep=';')

print("Archivos cargados correctamente.")

Archivos cargados correctamente.


# Crear funciones de preprocesamiento

In [9]:
# src/data/processing.py

import pandas as pd

def procesar_contaminante(df, nombre_col, decimal_coma=False):
    df = df.drop(columns=['Registros preliminares', 'Registros no validados', 'Unnamed: 5'], errors='ignore')
    if decimal_coma:
        df['Registros validados'] = df['Registros validados'].str.replace(',', '.').astype(float)
    else:
        df['Registros validados'] = pd.to_numeric(df['Registros validados'], errors='coerce')
    df['FECHA_HORA'] = pd.to_datetime(df['FECHA (YYMMDD)'].astype(str) + df['HORA (HHMM)'].astype(str).str.zfill(4), format='%y%m%d%H%M')
    df = df.set_index('FECHA_HORA')

    nulls = df[df['Registros validados'].isnull()]
    nulos_por_dia = nulls.groupby(nulls.index.date).size()
    dias_completos = nulos_por_dia[nulos_por_dia == 24].index.tolist()
    mask = [date not in dias_completos for date in df.index.date]
    df = df[mask]

    df['Registros validados'] = df['Registros validados'].interpolate(method='time')
    return df.rename(columns={'Registros validados': nombre_col})[[nombre_col]]

def procesar_meteorologia(df):
    df['date'] = pd.to_datetime(df['date'])
    df = df[['date', 'tmin', 'tavg', 'tmax', 'prcp', 'wspd', 'pres']]
    df = df.set_index('date')
    df = df.interpolate(method='time')
    return df

In [11]:
from src.data.processing import procesar_contaminante, procesar_meteorologia
import pandas as pd

# Carga de archivos
df_pm10 = pd.read_csv('../data/raw/pm 10 coyhaique (extended).csv', sep=';', encoding='latin1', engine='python')
df_pm25 = pd.read_csv('../data/raw/pm 2.5 coyhaique.csv', sep=';')
df_co = pd.read_csv('../data/raw/CO coyhaique.csv', sep=';')
df_so2 = pd.read_csv('../data/raw/SO2 coyahique.csv', sep=';')
df_meteo = pd.read_csv('../data/raw/export (extended).csv')
df_umbrales = pd.read_csv('../data/raw/umbrales_pm10.csv', sep=';')

# Procesamiento
df_pm10 = procesar_contaminante(df_pm10, 'PM10')
df_pm25 = procesar_contaminante(df_pm25, 'PM2.5')
df_co = procesar_contaminante(df_co, 'CO', decimal_coma=True)
df_so2 = procesar_contaminante(df_so2, 'SO2')
df_meteo = procesar_meteorologia(df_meteo)

ModuleNotFoundError: No module named 'src'

In [12]:
import sys
import os
import pandas as pd

# Ruta relativa desde el notebook a la carpeta src/
src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# 🔹 IMPORTAR FUNCIONES DEL MÓDULO DE PROCESAMIENTO
from data.processing import procesar_contaminante, procesar_meteorologia, unir_datasets

ImportError: cannot import name 'procesar_contaminante' from 'data.processing' (C:\Users\PC\Documents\Git-Hub\prediccion contaminacion del aire por calefaccion\calefaccion-contaminacion-aire-tdf\src\data\processing.py)